# Supermarkets translation

This notebook it's just a draft that we are keeping in case that we want to implement dynamic translation in the future.

We decided not to implement for time constrais and also because the IBM Watson was not as accurate as Google Translator.

In [ ]:
import pandas as pd
import numpy as np

# Import for the Translation with IBM Watson
import requests
import os
import json
import getpass
import ast # convert str to dic

# Imports the Google Cloud client library
# from googletrans import Translator

In [ ]:
#supermarkets_en = {
    #'aldi_north': pd.read_csv('data/products-en/aldinorth-products-en.csv'),
    #'aldi_south': pd.read_csv('data/products-en/aldisouth_products-en.csv'),
    #'edeka': pd.read_csv('data/products-en/edeka-products-en.csv'),
    #'kaufland': pd.read_csv('data/products-en/kaufland-products-en.csv'),
    #'lidl': pd.read_csv('data/products-en/lidl-products-en.csv'),
    #'rewe': pd.read_csv('data/products-en/rewe-product-en.csv')
#}

## 1. Translate
We have 2 options for dynamic translation (googletrans or IBM Watson) and 1 option for static translation (Google Translator)

We have decided to use google translate since the output was more accurate than what I got from IBM Watson. However, this is part of the code. In the case that in the future we need to do the translation dynamically. 

### 1.1 IBM Watson Translate API

In [ ]:
# Get the api key
api_key = getpass.getpass(prompt='IBM API key: ')

In [ ]:
url = "https://api.eu-de.language-translator.watson.cloud.ibm.com/instances/8218aa64-0979-48a3-9395-cefb38349b8e/v3/translate?version=2018-05-01"
headers = {"Content-Type": "application/json"}
auth = ("apikey", api_key)

In [ ]:
test = 'Pfanner Mango Fair 100%  Fair'
data = {"text":test,"model_id":"de-en"}

In [ ]:
json.dumps(data)

In [ ]:
# Request to the IBM Watson API
r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))

In [ ]:
r.json()

In [ ]:
r.json()['translations'][0]['translation']

In [ ]:
ast.literal_eval(json.dumps(data))['text']

IBM has a limitation of 'Exceeded max content length for translate request, max allowed is 50 KiB.'
So, in order to translate the whole csv file, we will have to go through each cell and translate it. We only need to translate 'Name', 'Unit' and 'Packet Size'.

In [ ]:
# Create an empty DataFrame for the tranlated ingredients with the 'Name' as key
supermarkets_example = {}
supermarkets_example['rewe-en'] = pd.DataFrame(columns=('Name', 'Name-en'))

In [ ]:
supermarkets_example['rewe-en']

In [ ]:
data = {"text":['Wasser'],"model_id":"de-en"}

json.dumps(data)

r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))
           
new_row = {'Name': ast.literal_eval(json.dumps(data))['text'][0], 'Name-en': r.json()['translations'][0]['translation']}

In [ ]:
new_row

In [ ]:
supermarkets_example['rewe-en'] = supermarkets_example['rewe-en'].append(new_row, ignore_index=True)

In [ ]:
supermarkets_example['rewe-en']

In [ ]:
[supermarkets['rewe']['Name'][1]]

In [ ]:
src = 'de' # Source language
dest = 'en' # Destiny language

supermarkets_en = {
    'aldi_north': pd.DataFrame(columns=('Name', 'Name-en')),
    'aldi_south': pd.DataFrame(columns=('Name', 'Name-en')),
    'edeka': pd.DataFrame(columns=('Name', 'Name-en')),
    'kaufland': pd.DataFrame(columns=('Name', 'Name-en')),
    'lidl': pd.DataFrame(columns=('Name', 'Name-en')),
    'rewe': pd.DataFrame(columns=('Name', 'Name-en'))
}

In [ ]:
for market in supermarkets:
    # Create new dataframe for the english version
    # supermarkets[f'{market}-en'] = supermarkets[market].copy()

    # Loop through each row
    for row in range(0,5):# range(len(supermarkets[market])):
        try:
            data = {"text":[supermarkets[market]['Name'][row]], "model_id":"de-en"}
            r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))
            # Create a new row with the word in 'de' and in 'en'
            new_row = {'Name': ast.literal_eval(json.dumps(data))['text'][0], 'Name-en': r.json()['translations'][0]['translation']}
            supermarkets_en[market] = supermarkets[market].append(new_row, ignore_index=True)

        except:
            print(supermarkets[market]['Name'][row])

In [ ]:
supermarkets_en['rewe']

In [ ]:
data = {"text":test,"model_id":"de-en"}
json.dumps(data)

# Request to the IBM Watson API
r = requests.post(headers=headers, auth=auth, url=url, data=json.dumps(data))

r.json()

r.json()['translations'][0]['translation']

In [ ]:
The method accepts lists as input as well. So we are going to loop per each row in order to catch the rows that raise errors

### 1.2 Googletrans

In [ ]:
# Instantiates a client
translator = Translator()

In [ ]:
# Example to see how it works
# 1 - translator.translate('Hello').text

In [ ]:
# 1 - word.origin # .origin -> prints the original word

In [ ]:
# 1 - word[0].text # .text -> prints the translated word

In [ ]:
# Undertand how the method works
# 1 - translator.translate(supermarkets['rewe']['Name'][1], dest='en').text